# Building a chatbot Q/A over the Rare As One Cycle 1 & 2 literature.  

> Pragmatic tools for constructing databases of scientific works based on queries defined with Boolean Logic.

## Preliminaries

Here we set up libraries and methods to create and query the local Postgres database we will be using to store our information from the Alhazen tools and agent

In [1]:
from alhazen.core import OllamaRunner
from alhazen.schema_sqla import * 
from alhazen.tools.basic import *
from alhazen.tools.metadata_extraction_tool import *
from alhazen.toolkit import AlhazenToolkit
from alhazen.utils.jats_text_extractor import NxmlDoc
from alhazen.utils.ceifns_db import Ceifns_LiteratureDb, create_ceifns_database, drop_ceifns_database

from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector

from bs4 import BeautifulSoup,Tag,Comment,NavigableString
from databricks import sql
from datetime import datetime, timedelta
from importlib_resources import files
import os
import pandas as pd
from pathlib import Path
import re
import requests
from sqlalchemy import create_engine, exists, func
from sqlalchemy.orm import sessionmaker, aliased

from time import time, sleep
from tqdm import tqdm
from urllib.request import urlopen
from urllib.parse import quote_plus, quote, unquote
from urllib.error import URLError, HTTPError
import yaml

In [2]:
# Using Aliases like this massively simplifies the use of SQLAlchemy
IR = aliased(InformationResource)

SKC = aliased(ScientificKnowledgeCollection)
SKC_PROV = aliased(ScientificKnowledgeCollectionProvenance)
SKC_HM = aliased(ScientificKnowledgeCollectionHasMembers)
SKE = aliased(ScientificKnowledgeExpression)
SKE_XREF = aliased(ScientificKnowledgeExpressionXref)
SKE_HR = aliased(ScientificKnowledgeExpressionHasRepresentation)
SKI = aliased(ScientificKnowledgeItem)
SKI_HP = aliased(ScientificKnowledgeItemHasPart)
SKF = aliased(ScientificKnowledgeFragment)

N = aliased(Note)
SKC_HN = aliased(ScientificKnowledgeCollectionHasNotes)
SKE_HN = aliased(ScientificKnowledgeExpressionHasNotes)
SKI_HN = aliased(ScientificKnowledgeItemHasNotes)
SKF_HN = aliased(ScientificKnowledgeFragmentHasNotes)

Remember to set environmental variables for this code:

* `ALHAZEN_DB_NAME` - the name of the Postgres database you are storing information into
* `LOCAL_FILE_PATH` - the location on disk where you save files for your digital library, downloaded models or other data.   

In [3]:
os.environ['ALHAZEN_DB_NAME'] = 'rare_as_one_diseases'

# Set this path to the location on disk that you'd like to store all Alhazen related files. This
os.environ['LOCAL_FILE_PATH'] = '/users/gully.burns/alhazen/'

if os.path.exists(os.environ['LOCAL_FILE_PATH']) is False:
    os.makedirs(os.environ['LOCAL_FILE_PATH'])

In [4]:
if os.environ.get('ALHAZEN_DB_NAME') is None: 
    raise Exception('Which database do you want to use for this application?')
db_name = os.environ['ALHAZEN_DB_NAME']

if os.environ.get('LOCAL_FILE_PATH') is None: 
    raise Exception('Where are you storing your local literature database?')
loc = os.environ['LOCAL_FILE_PATH']

Run this command to destroy your current database 

**USE WITH CAUTION**

In [13]:
drop_ceifns_database(os.environ['ALHAZEN_DB_NAME'])

Database has been dropped successfully !!


Run this command to create a new, empty database. 

In [14]:
create_ceifns_database(os.environ['ALHAZEN_DB_NAME'])

100%|██████████| 315/315 [00:00<00:00, 3446.51it/s]


This command lists all the tools the Alhazen agent system has access to

In [15]:
ldb = Ceifns_LiteratureDb(loc=loc, name=db_name)
ollr = OllamaRunner('mixtral')
llm  = ollr.llm

tk = AlhazenToolkit(db=ldb, ollr=ollr)
print('AVAILABLE TOOLS')
for t in tk.get_tools():
    print('\t'+type(t).__name__)

AVAILABLE TOOLS
	AddCollectionFromEPMCTool
	DescribeCollectionCompositionTool
	DeleteCollectionTool
	RetrieveFullTextTool
	MetadataExtractionTool


# Build paper collections

This section will build a literature collection across each of the diseases in the Rare As One Cohorts for cycle 1 and 2. 



In [16]:
os.environ['NCBI_API_KEY'] = 'd086451c882fabace54d7b049b6fb8481908'

What diseases are we querying the literature for?

In [18]:
import local_resources.data_files.rao_grantees as rao_files
from alhazen.utils.queryTranslator import QueryTranslator, QueryType

cols_to_include = ['ID', 'CORPUS_NAME', 'TERMS']
df = pd.read_csv(files(rao_files).joinpath('CZI_RAO_diseases.tsv'), sep='\t')
df = df.drop(columns=[c for c in df.columns if c not in cols_to_include])

df

,ID,CORPUS_NAME,TERMS
0,1,Adult Polyglucosan Body Disease,adult polyglucosan body disease | adult polygl...
1,2,Creatine transporter deficiency,creatine transporter deficiency | guanidinoace...
2,3,AGAT deficiency,"GATM deficiency | ""AGAT deficiency"" | ""arginin..."
3,4,Guanidinoacetate methyltransferase deficiency,guanidinoacetate methyltransferase deficiency ...
4,5,CLOVES Syndrome,CLOVES syndrome | (congenital lipomatous overg...
...,...,...,...
76,78,TBCK Syndrome,TBCK Syndrome | TBCK Encephalopathy | TBCK-ass...
77,79,Dyskeratosis congenita,dyskeratosis congenita | Zinsser-Engman-Cole s...
78,80,Telomere syndrome,telomere syndrome | short telomere syndrome
79,81,The Stiff Person Syndrome,stiff man syndrome | stiff person syndrome | M...


This command iterates over the list of different collections and runs a query for each one on the European website by processing the `TERMS` column from the  dataframe with the `QueryTranslator` utility. This generates a search query in boolean logic that searches the `TITLE_ABS` field in the remote database (See https://www.ebi.ac.uk/europepmc/webservices/rest/fields for possible fields to search).

In [19]:
qt = QueryTranslator(df.sort_values('ID'), 'ID', 'TERMS', 'CORPUS_NAME')
(corpus_ids, epmc_queries) = qt.generate_queries(QueryType.epmc, sections=['TITLE_ABS'])
corpus_names = df['CORPUS_NAME']

addEMPCCollection_tool = [t for t in tk.get_tools() if isinstance(t, AddCollectionFromEPMCTool)][0]
for (id, name, query) in zip(corpus_ids, corpus_names, epmc_queries):
    #if id != 81:
    #    continue
    addEMPCCollection_tool.run(tool_input={'id': id, 'name':name, 'query':query, 'full_text':False})

100%|██████████| 81/81 [00:00<00:00, 16096.78it/s]


https://www.ebi.ac.uk/europepmc/webservices/rest/search?format=JSON&pageSize=1000&synonym=TRUE&resultType=core&query=((TITLE_ABS:"adult polyglucosan body disease") OR (TITLE_ABS:"adult polyglucosan body neuropathy")), 104 European PMC PAPERS FOUND


100%|██████████| 1/1 [00:03<00:00,  3.12s/it]


 Returning 92


100%|██████████| 92/92 [00:00<00:00, 508.47it/s]


...

100%|██████████| 902/902 [00:01<00:00, 500.25it/s]


https://www.ebi.ac.uk/europepmc/webservices/rest/search?format=JSON&pageSize=1000&synonym=TRUE&resultType=core&query=((TITLE_ABS:"isolated pontocerebellar hypoplasia") OR (TITLE_ABS:"nonsyndromic pontocerebellar hypoplasia") OR (TITLE_ABS:"pontocerebellar hypoplasia") OR (TITLE_ABS:"pontoneocerebellar atrophy") OR (TITLE_ABS:"pontoneocerebllar hypoplasia")), 380 European PMC PAPERS FOUND


100%|██████████| 1/1 [00:05<00:00,  5.56s/it]


 Returning 366


100%|██████████| 366/366 [00:00<00:00, 487.91it/s]


Query the database for the numbers of papers returned

In [20]:
q = ldb.session.query(SKC.id, SKC.name, func.count(SKC_HM.has_members_id)) \
    .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
    .group_by(SKC.id, SKC.name) \
    .order_by(SKC.id.cast(Integer))
corpora_df = pd.DataFrame(q.all(), columns=['Corpus ID', 'Corpus Name', 'Paper Count'])

paper_count = ldb.session.query(func.count(SKE.id)).first()
print('Count of all papers in database: %d'%(paper_count[0]))

corpora_df

Count of all papers in database: 64861


,Corpus ID,Corpus Name,Paper Count
0,1,Adult Polyglucosan Body Disease,92
1,2,Creatine transporter deficiency,300
2,3,AGAT deficiency,34
3,4,Guanidinoacetate methyltransferase deficiency,129
4,5,CLOVES Syndrome,94
...,...,...,...
76,78,TBCK Syndrome,8
77,79,Dyskeratosis congenita,1015
78,80,Telomere syndrome,44
79,81,The Stiff Person Syndrome,901


In [10]:
ldb.session.rollback()

In [24]:
ft_retriever  = [t for t in tk.get_tools() if isinstance(t, RetrieveFullTextTool)][0]

for i, c in corpora_df.iterrows():
    if c['Corpus ID'] != '81':
        continue
    print(c['Corpus Name'])
    ft_count = 0
    no_ft_count = 0
    doi_list = [e.id for e in ldb.list_expressions(collection_id=c['Corpus ID'])]
    for doi in doi_list:
        d2 = doi.replace('doi:', '')
        path = loc+db_name+'/ft/'
        nxml_file_path = path+'/'+d2+'.nxml'
        pdf_file_path = path+'/'+d2+'.pdf'
        html_file_path = path+'/'+d2+'.html'
        if os.path.exists(nxml_file_path) or  \
                os.path.exists(pdf_file_path) or \
                os.path.exists(html_file_path):
            ft_count += 1
        try: 
            no_ft_count += 1
            #print('\t'+doi)
            ft_retriever.run(tool_input={'paper_id': doi})
        except Exception as e:
            print(e)
    print(ft_count)
    print(no_ft_count)

The Stiff Person Syndrome
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key=d086451c882fabace54d7b049b6fb8481908&db=pmc&term=013163/aim.0016[doi]&retmode=xml
No paper found with that DOI
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key=d086451c882fabace54d7b049b6fb8481908&db=pmc&term=013163/aim.0016[doi]&retmode=xml
No paper found with that DOI
min() arg is an empty sequence
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key=d086451c882fabace54d7b049b6fb8481908&db=pmc&term=10.1001/archneur.1960.00450040098012[doi]&retmode=xml
No paper found with that DOI
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key=d086451c882fabace54d7b049b6fb8481908&db=pmc&term=10.1001/archneur.1960.00450040098012[doi]&retmode=xml
No paper found with that DOI
min() arg is an empty sequence
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key=d086451c882fabace54d7b049b6fb8481908&db=pmc&term=10.1001/archneur.1971.00480310050004[doi]&

KeyboardInterrupt: 

In [28]:
q = ldb.session.query(SKE.id, SKI.id, SKI.type, SKF.id, SKF.type, SKF.offset, SKF.content) \
    .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
    .filter(SKC_HM.has_members_id==SKE.id) \
    .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
    .filter(SKE_HR.has_representation_id==SKI.id) \
    .filter(SKI.id==SKI_HP.ScientificKnowledgeItem_id) \
    .filter(SKI_HP.has_part_id==SKF.id) \
    .filter(SKE_HR.has_representation_id==SKI.id) \
    .filter(SKF.type=='section') \
    .filter(SKI.type.like('%FullText')) \
    .order_by(SKE.id, SKF.offset)
items_df = pd.DataFrame(q.all(), columns=['doi', 'item_id', 'item_type', 'fragment_id', 'fragment_type', 'offset', 'content'])

items_df

,doi,item_id,item_type,fragment_id,fragment_type,offset,content
0,doi:10.1001/archinte.1994.00420110133015,7e6a350d2d,PDFFullText,7e6a350d2d.0,section,0,•\n
1,doi:10.1001/archinte.1994.00420110133015,7e6a350d2d,PDFFullText,7e6a350d2d.1,section,2,Triple Threat Broad Eligibility\n
2,doi:10.1001/archinte.1994.00420110133015,7e6a350d2d,PDFFullText,7e6a350d2d.2,section,36,Concerns\n
3,doi:10.1002/ccr3.2538,36af029190,JATSFullText,36af029190.0,section,2727,1\nINTRODUCTION\nAutoantibodies to glutamic ac...
4,doi:10.1002/ccr3.2538,36af029190,JATSFullText,36af029190.1,section,3532,2\nCASE REPORT\nA 61-year-old right-handed wom...
...,...,...,...,...,...,...,...
682,doi:10.3389/fnmol.2018.00291,9e6ac0056e,JATSFullText,9e6ac0056e.19,section,81666,Mouse Models With Defective Trafficking of Gly...
683,doi:10.3389/fnmol.2018.00291,9e6ac0056e,JATSFullText,9e6ac0056e.20,section,94537,Outlook\nThis review summarizes the traffickin...
684,doi:10.3389/fnmol.2018.00291,9e6ac0056e,JATSFullText,9e6ac0056e.21,section,97042,Ethics Statement\nExperiments were approved by...
685,doi:10.3389/fnmol.2018.00291,9e6ac0056e,JATSFullText,9e6ac0056e.22,section,97315,"Author Contributions\nNS, VR, and CV performed..."


# Index the abstracts and run some simple semantic queries

Here we index each paper's title and abstract to build a simple question / answer interface.

In [15]:
ldb.session.rollback()

In [32]:
for i, c in tqdm(corpora_df.iterrows()):
    if c['Corpus ID'] != '81':
        continue
    expressions = ldb.list_expressions(collection_id=c['Corpus ID'])    
    ldb.embed_expression_list(expressions)

81it [01:27,  1.07s/it]


In [21]:
question = 'What is known about genetics underlying Stiff Person Syndrome?'

ldb.query_vectorindex(question, k=10, collection_name='ScienceKnowledgeItem_FullText')

[(Document(page_content='Introduction\nStiff-person syndrome (SPS) is an uncommon disorder characterized by progressive stiffness, rigidity, and painful spasm affecting axial muscle. It can lead to significant debilitation and affects ambulation. Usually, it is associated with autoimmunity as it has a significant overlap with autoantibody in type 1 diabetes. Hypopituitarism, especially hypocortisolism, can lead to axial muscle stiffness and rigidity, similar to an SPS. This case highlights a patient with pituitary adenoma and panhypopituitarism with a stiff person-like syndrome as the initial presentation.', metadata={'c_ids': '81', 'e_id': 'doi:10.1159/000522253', 'e_type': 'ClinicalCaseReport', 'i_id': 'a427205ffe', 'i_type': 'JATSFullText', 'f_id': 'a427205ffe.0', 'citation': 'Goh KG, Yusof Khan AHK, Nasruddin A. (2022) Stiff Person-Like Syndrome: An Unusual Presentation of Pituitary Macroadenoma with Panhypopituitarism.'}),
  0.11609077453613281),
 (Document(page_content='1\nINTROD

## ATTEMPTING TO RECONSTRUCT PAPER-QA PIPELINE IN OUR SYSTEM.

1. Embed paper sections + question
2. Given the question, summarize the retrieved paper sections relative to the question
3. Score and select relevant passages
4. Put summaries into prompt
5. Generate answer with prompt


In [30]:
 
os.environ['PGVECTOR_CONNECTION_STRING'] = "postgresql+psycopg2:///"+ldb.name
vectorstore = PGVector.from_existing_index(
        embedding = ldb.embed_model, 
        collection_name = 'ScienceKnowledgeItem') 
retriever = vectorstore.as_retriever(search_kwargs={'k':15, 'filter': {'skc_ids': 81}})
#retriever = vectorstore.as_retriever()


In [31]:
retriever.invoke(question)

[Document(page_content='[Stiff-person syndrome: a clinical observation].\n\nStiff-person syndrome (SPS) is a rare chronic neurological disease characterized by progressing muscle rigidity and painful muscle spasms. The signs of SPS are pain and stiffness in spinal, abdominal and cervical muscles, increased muscle tonus in extensor muscles of extremities, constant stiffness of paravertebral and abdominal muscles and muscle spasms. A clinical case of a SPS patient T., aged 23 years, is presented. The peculiarity of this case is additional left-sided peripheral upper extremity monoparesis, which is most likely associated with the development of left-sided compression-ischemic brachial plexopathy resulted from profound muscular tonic syndrome in the neck and shoulder girdles.', metadata={'skc_ids': '81', 'ske_id': 'doi:10.17116/jnevro201911906196', 'ski_id': 'fb81803b8f', 'skf_id': 'fb81803b8f.0', 'type': 'CitationRecord', 'citation': 'Isaeva NV, Prokopenko SV, Rodikov MV, Abroskina MV, On

In [12]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel

In [29]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain.chat_models import ChatOllama
from langchain.schema import get_buffer_string, OutputParserException, format_document
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from alhazen.utils.output_parsers import JsonEnclosedByTextOutputParser

#from paperqa.prompts import summary_prompt as paperqa_summary_prompt, qa_prompt as paperqa_qa_prompt, select_paper_prompt, citation_prompt, default_system_prompt

hum_p = '''First, read through the following JSON encoding of {k} research articles: 

Each document has three attributes: (A) a digital object identifier ('DOI') code, (B) a CITATION string containing the authors, publication year, title and publication location, and the (C) CONTENT field with the title and abstract of the paper.  

```json:{context}```

Then, generate a JSON list of summaries of each article in order to help answer the following question:

Question: {question}

Do NOT directly answer the question, instead summarize to give evidence to help answer the question. 
Focus on specific details, including numbers, equations, or specific quotes. 
Reply "Not applicable" if text is irrelevant. 
Restrict each summary to {summary_length} words. 
Also, provide a score from 1-10 indicating relevance to question. Do not explain your score. 

Write this answer as JSON formatted output. Provide a list of {k} dict objects with the following fields: DOI, SUMMARY, RELEVANCE SCORE. 

Do not provide additional explanation for the answer.
Do not include any other response other than a JSON object.
'''
sys_p = '''Answer in a direct and concise tone. Your audience is an expert, so be highly specific. If there are ambiguous terms or acronyms, first define them.'''

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="'DOI': '{ske_id}', CITATION: '{citation}', CONTENT:'{page_content}'")
def combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="},{\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return '[{'+document_separator.join(doc_strings)+'}]'

template = ChatPromptTemplate.from_messages([
            ("system", sys_p),
            ("human", hum_p)])

qa_chain = (
    RunnableParallel({
        "k": itemgetter("k"),
        "question": itemgetter("question"),
        "summary_length": itemgetter("summary_length"),
        "context": itemgetter("question") | retriever | combine_documents,
    })
    | {
        "summary": template | ChatOllama(model='mixtral') | JsonEnclosedByTextOutputParser(),
        "context": itemgetter("context"),
    }
)

input = {'question': question, 'summary_length': 1000, 'k':5}    
out = qa_chain.invoke(input, config={'callbacks': [ConsoleCallbackHandler()]})
print(json.dumps(out, indent=4))




[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What clinical indicators are present for patients suffering from stiff person syndrome (SPS)?",
  "summary_length": 1000,
  "k": 5
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "question": "What clinical indicators are present for patients suffering from stiff person syndrome (SPS)?",
  "summary_length": 1000,
  "k": 5
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableLambda] Entering Chain run with input:
{
  "question": "What clinical indicators are present for patients suffering from stiff person syndrome (SPS)?",
  "summary_length": 1000,
  "k": 5
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": 5
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 4:chain:RunnableLambda] Entering Chain run 

# Run Fast HuggingFace Deep Learning Tools over corpus 

In [80]:
from transformers import pipeline, AutoModel, AutoTokenizer
import torch

model_path = '/Users/gully.burns/Documents/2024H1/models/discourse_tagger'
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1", 
                                          truncation=True, 
                                          max_length=512)
labels = ['BACKGROUND', 'OBJECTIVE', 'METHODS', 'RESULTS', 'CONCLUSIONS']
lookup = {'LABEL_%d'%(i):l for i, l in enumerate(labels)}
model = AutoModel.from_pretrained(model_path)
model.eval()

classifier = pipeline("text-classification", 
                      model = model_path, 
                      tokenizer=tokenizer, 
                      truncation=True,
                      batch_size=8,
                      device='mps')


In [85]:
# Try an out-of-the-box classifier on the data for discourse tagging.
from transformers import pipeline

ldb.session.rollback()
one_year_ago = (datetime.now() - timedelta(days=1*365))

q = ldb.session.query(SKE, SKF) \
    .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
    .filter(SKC_HM.has_members_id==SKE.id) \
    .filter(SKE.id==SKE_HR.ScientificKnowledgeExpression_id) \
    .filter(SKE_HR.has_representation_id==SKI.id) \
    .filter(SKI.id==SKI_HP.ScientificKnowledgeItem_id) \
    .filter(SKI_HP.has_part_id==SKF.id) \
    .filter(SKE_HR.has_representation_id==SKI.id) \
    .filter(SKI.type == 'CitationRecord' ) \
    .order_by(SKE.id)

#   .filter(SKC.name == 'The Stiff Person Syndrome' ) \
#   .filter(SKE.publication_date >= one_year_ago) \

s_list = []
for e, f in q.all():
    for i, s in enumerate(ldb.sent_detector.tokenize(f.content)):
        s_list.append([e.id, f.id, i, s])
sent_df = pd.DataFrame(s_list, columns=['doi', 'f_id', 's_id', 'text'])
sent_df

,doi,f_id,s_id,text
0,doi:/s0034-98872008000200015,9f3919db23.0,0,[Cholangiocarcinoma].
1,doi:/s0034-98872008000200015,9f3919db23.1,0,Cholangiocarcinoma is a malignant lesion of th...
2,doi:/s0034-98872008000200015,9f3919db23.1,1,Its incidence and prevalence are low.
3,doi:/s0034-98872008000200015,9f3919db23.1,2,It appears from the sixth decade of life and t...
4,doi:/s0034-98872008000200015,9f3919db23.1,3,It is most frequently found in the confluence ...
...,...,...,...,...
538367,doi:huon.2005.49.1.0065,9f08f2e243.1,9,Of the 59 primary epithelial tumours 62.7% was...
538368,doi:huon.2005.49.1.0065,9f08f2e243.1,10,The differential diagnosis and management are ...
538369,doi:huon.2005.49.1.0065,9f08f2e243.1,11,The prognosis of pleomorphic adenomas depends ...
538370,doi:huon.2005.49.1.0065,9f08f2e243.1,12,In cases of suspected malignant epithelial tum...


In [86]:

# Predict multipe texts on single CPU and time the inference duration
start = time()

df = sent_df

preds = classifier([row.text for i, row in df.iterrows()])
pred_df = pd.DataFrame(preds)
df['label'] = [lookup[row.label] for i, row in pred_df.iterrows()]
df['score'] = [row.score for i, row in pred_df.iterrows()]

end = time()

print('Prediction time:', str(timedelta(seconds=end-start)))

Prediction time: 0:26:27.240843


In [100]:
df

,doi,f_id,s_id,text,label,score
0,doi:/s0034-98872008000200015,9f3919db23.0,0,[Cholangiocarcinoma].,BACKGROUND,0.556335
1,doi:/s0034-98872008000200015,9f3919db23.1,0,Cholangiocarcinoma is a malignant lesion of th...,BACKGROUND,0.710530
2,doi:/s0034-98872008000200015,9f3919db23.1,1,Its incidence and prevalence are low.,BACKGROUND,0.769202
3,doi:/s0034-98872008000200015,9f3919db23.1,2,It appears from the sixth decade of life and t...,BACKGROUND,0.772743
4,doi:/s0034-98872008000200015,9f3919db23.1,3,It is most frequently found in the confluence ...,BACKGROUND,0.726111
...,...,...,...,...,...,...
538367,doi:huon.2005.49.1.0065,9f08f2e243.1,9,Of the 59 primary epithelial tumours 62.7% was...,RESULTS,0.916542
538368,doi:huon.2005.49.1.0065,9f08f2e243.1,10,The differential diagnosis and management are ...,RESULTS,0.558720
538369,doi:huon.2005.49.1.0065,9f08f2e243.1,11,The prognosis of pleomorphic adenomas depends ...,BACKGROUND,0.629027
538370,doi:huon.2005.49.1.0065,9f08f2e243.1,12,In cases of suspected malignant epithelial tum...,BACKGROUND,0.707331


In [110]:
ldb.session.rollback()

In [111]:
# Generate fragment sentences and add them as Notes
ldb.session.rollback()
for i, row in df.iterrows():
    f_q = ldb.session.query(SKF).filter(SKF.id == row.f_id).first()
    i_q = ldb.session.query(SKI).filter(SKI.id == row.f_id.split('.')[0]).first()
    o = i_q.content.find(row.text)
    l = len(row.text)
    sentence_fragment = ScientificKnowledgeFragment(id=f_q.id+'.'+str(row.s_id), \
                                                    content=row.text, \
                                                    offset=o, \
                                                    length=l, \
                                                    type='sentence')
    i_q.has_part.append(sentence_fragment)
    note_content = {'discourse_label': row.label, 'score': row.score}
    n = Note(id=f_q.id+'.'+str(row.s_id)+'.discourse_type',
             content=json.dumps(note_content, indent=4),
             format='json',
             type='NoteAboutFragment')
    sentence_fragment.has_notes.append(n)
    ldb.session.flush()
ldb.session.commit()


# Running DRSM Classifiers.

In [ ]:
from transformers import pipeline, AutoModel, AutoTokenizer
import torch

model_path = '/Users/gully.burns/Documents/2024H1/models/drsm_classifier'
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1", 
                                          truncation=True, 
                                          max_length=512)
labels = ['BACKGROUND', 'OBJECTIVE', 'METHODS', 'RESULTS', 'CONCLUSIONS']
lookup = {'LABEL_%d'%(i):l for i, l in enumerate(labels)}
model = AutoModel.from_pretrained(model_path)
model.eval()

classifier = pipeline("text-classification", 
                      model = model_path, 
                      tokenizer=tokenizer, 
                      truncation=True,
                      batch_size=8,
                      device='mps')


# Topic Modeling over the corpus. 

What are the main topics being discussed in each paper?

In [ ]:
from transformers import pipeline, AutoModel, AutoTokenizer
import torch

model_path = '/Users/gully.burns/Documents/2024H1/models/drsm_classifier'
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1", 
                                          truncation=True, 
                                          max_length=512)
labels = ['BACKGROUND', 'OBJECTIVE', 'METHODS', 'RESULTS', 'CONCLUSIONS']
lookup = {'LABEL_%d'%(i):l for i, l in enumerate(labels)}
model = AutoModel.from_pretrained(model_path)
model.eval()

classifier = pipeline("text-classification", 
                      model = model_path, 
                      tokenizer=tokenizer, 
                      truncation=True,
                      batch_size=8,
                      device='mps')

# Search for and download Full Text Papers.

Can we search for all Stiff Person Syndrome papers published in the last 10 years?



In [44]:
ldb.session.rollback()

ten_years_ago = (datetime.now() - timedelta(days=10*365))
print(ten_years_ago)

q = ldb.session.query(func.extract('year', SKE.publication_date.cast(Date)), func.count(SKE.id) ) \
    .filter(SKC.id==SKC_HM.ScientificKnowledgeCollection_id) \
    .filter(SKC_HM.has_members_id==SKE.id) \
    .filter(SKE.publication_date >= ten_years_ago) \
    .filter(SKC.name == 'The Stiff Person Syndrome' ) \
    .group_by(func.extract('year', SKE.publication_date.cast(Date))) \
    .order_by(func.extract('year', SKE.publication_date.cast(Date)))
sps_pubcount_df = pd.DataFrame(q.all(), columns=['doi', 'date'])
sps_pubcount_df

2014-01-25 10:10:34.434342


,doi,date
0,2014,25
1,2015,35
2,2016,34
3,2017,26
4,2018,24
5,2019,43
6,2020,40
7,2021,40
8,2022,35
9,2023,49
